In [ ]:
test = pd.read_parquet('totalPlayers.parquet',engine='pyarrow')
linksUnique = pd.read_parquet('allPlayersLink.parquet')['PlayersLink'].unique().tolist()
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
index=0
for linkP in linksUnique:
    id1=linkP.split('profil/spieler/')[1]
     name=test[test['playersId']==id1]['playersName'].iloc[0]
    link = 'https://www.transfermarkt.com'+linkP
    page = requests.get(link, headers=headers)
    pageSoup = BeautifulSoup(page.content , 'html.parser')
    print(id1)
    TransferData = pageSoup.find_all('div',{'class':'box viewport-tracking'})[1]
    season  = TransferData.find_all('div',{'class':'grid__cell grid__cell--center tm-player-transfer-history-grid__season'})
    date = TransferData.find_all('div',{'class':'grid__cell grid__cell--center tm-player-transfer-history-grid__date'})
    left = TransferData.find_all('div',{'class':'grid__cell grid__cell--center tm-player-transfer-history-grid__old-club'})
    joined = TransferData.find_all('div',{'grid__cell grid__cell--center tm-player-transfer-history-grid__new-club'})
    mv = TransferData.find_all('div',{'class':'grid__cell grid__cell--center tm-player-transfer-history-grid__market-value'})
    fee = TransferData.find_all('div',{'class':'grid__cell grid__cell--center tm-player-transfer-history-grid__fee'})
    playersId , playersName , season_ , date_ , left_,joined_, mv_ , fee_ = [],[],[],[],[],[],[],[]

    for i in range(len(season)):
        playersId.append(id1)
        playersName.append(name)
        season_.append(season[i].text.strip())
        date_.append(date[i].text.strip())
        left_.append([left[i].text.strip(),left[i].a.get('href').split('transfers/verein/')[1].split('/')[0]])
        joined_.append([joined[i].text.strip(),joined[i].a.get('href').split('transfers/verein/')[1].split('/')[0]])
        mv_.append(mv[i].text.strip())
        fee_.append(fee[i].text.strip())
    if(index==0):
      df = pd.DataFrame({'playersId':playersId,'playersName':playersName,'season':season_ , 'date':date_ , 'left':left_ , 'joined':joined_ , 'mv':mv_ , 'fee':fee_})

    else:
      df1 = pd.DataFrame({'playersId':playersId,'playersName':playersName,'season':season_ , 'date':date_ , 'left':left_ , 'joined':joined_ , 'mv':mv_ , 'fee':fee_})
      df1=pd.concat([df1,df],axis=0)
    index+=1

df1.reset_index(drop=True).to_parquet('PlayerTransfer.parquet',engine='pyarrow')
df1.reset_index(drop=True).to_csv('PlayerTransfer')
